In [1]:
import sys
sys.path.append(r'C:\Users\kuina\OneDrive\TFG\Codigo')

In [2]:
from datasets import load_dataset
import json

from utils_creacion_datasets import create_dependency_tags

In [3]:
map_about = {0:'ABOUT:female',1:'ABOUT:male'}
rev_about = {v:k for k,v in map_about.items()}
map_self = {0:'SELF:female',1:'SELF:male'}
map_partner = {0:'PARTNER:female',1:'PARTNER:male'}

In [4]:
list(map(map_about.get,[0,0,1]))

['ABOUT:female', 'ABOUT:female', 'ABOUT:male']

## Datos Huggingface

In [5]:
convai2_about = load_dataset('md_gender_bias','convai2_inferred')
train_about = convai2_about['train']
print(train_about[0])

Reusing dataset md_gender_bias (C:\Users\kuina\.cache\huggingface\datasets\md_gender_bias\convai2_inferred\1.0.0\8ae77b51acf93383161cc954b146159291beca6c979b54ce228c46db86116c05)


  0%|          | 0/3 [00:00<?, ?it/s]

{'text': "hi , how are you doing ? i'm getting ready to do some cheetah chasing to stay in shape .", 'binary_label': 1, 'binary_score': 0.6521999835968018, 'ternary_label': 2, 'ternary_score': 0.4496000111103058}


In [6]:
train_about = train_about.remove_columns(['binary_score', 'ternary_label', 'ternary_score'])

In [7]:
train_about

Dataset({
    features: ['text', 'binary_label'],
    num_rows: 131438
})

In [8]:
#Filtro repetidos para que sean una única entrada (Aqui aun no estoy filtrando las etiquetas)
data = {}
print(f'Elementos antes de filtrar repetidos = {len(train_about)}')
for item in train_about:
    text = item['text']
    label = item['binary_label']
    if text not in data:
        # data[text] =[label]
        data[text] =[map_about[label]]
    else:
        data[text].append(map_about[label])

print(f'Elementos después de filtrar repetidos = {len(data)}')

Elementos antes de filtrar repetidos = 131438
Elementos después de filtrar repetidos = 115877


In [10]:
# Filtro las etiquetas:

for item in data:
    data[item] = list(set(data[item]))


In [11]:
for item in data:
    print(data[item])
    break

['ABOUT:male']


## Carga datos ParlAI

In [12]:
with open(r'./convai_two_dataset.json') as f: #Este tiene unks
    convai2_parlai = json.load(f)

In [13]:
convai2_parlai

{"good choice . i haven't figured out what i want to do yet .": {'partner': ['PARTNER:unknown'],
  'self': ['SELF:unknown']},
 "that's ok , i also love to watch horror shows ; do you ?": {'partner': ['PARTNER:male'],
  'self': ['SELF:unknown']},
 'i like science as well': {'self': ['SELF:unknown'],
  'partner': ['PARTNER:unknown']},
 'i just crunched up halloween oreos into some ice cream . pretty good .': {'self': ['SELF:unknown'],
  'partner': ['PARTNER:unknown']},
 "sometimes i do . while i'm asleep": {'partner': ['PARTNER:unknown'],
  'self': ['SELF:male']},
 'haha ! just do not let her hear you say that !': {'self': ['SELF:male'],
  'partner': ['PARTNER:male']},
 'ouch , stay safe ! hows the traffic there ?': {'self': ['SELF:unknown'],
  'partner': ['PARTNER:unknown']},
 'well , i do work at a party store ! love to party !': {'self': ['SELF:unknown'],
  'partner': ['PARTNER:unknown']},
 "ya they don't like that i believe in any a .": {'self': ['SELF:unknown'],
  'partner': ['PARTN

# Unión de los datasets

Itero sobre convai porque tiene menos elementos (no deberia hacerme falta comprobar que ya los haya incluido porque es algo que tuve en cuenta con los datasets individuales, ahroa solo tengo qeu iterar sobre uno y ver si la frase tiene etiqueta en el otro)

In [14]:
join_data = {}
for item in convai2_parlai:
    if item in data:
        join_data[item] = {'label_about':data[item],
                            'label_to':convai2_parlai[item]['partner'],
                            'label_as':convai2_parlai[item]['self']}


In [15]:
join_data

{"good choice . i haven't figured out what i want to do yet .": {'label_about': ['ABOUT:female'],
  'label_to': ['PARTNER:unknown'],
  'label_as': ['SELF:unknown']},
 "that's ok , i also love to watch horror shows ; do you ?": {'label_about': ['ABOUT:male'],
  'label_to': ['PARTNER:male'],
  'label_as': ['SELF:unknown']},
 'i like science as well': {'label_about': ['ABOUT:male'],
  'label_to': ['PARTNER:unknown'],
  'label_as': ['SELF:unknown']},
 'i just crunched up halloween oreos into some ice cream . pretty good .': {'label_about': ['ABOUT:female'],
  'label_to': ['PARTNER:unknown'],
  'label_as': ['SELF:unknown']},
 "sometimes i do . while i'm asleep": {'label_about': ['ABOUT:female'],
  'label_to': ['PARTNER:unknown'],
  'label_as': ['SELF:male']},
 'haha ! just do not let her hear you say that !': {'label_about': ['ABOUT:female'],
  'label_to': ['PARTNER:male'],
  'label_as': ['SELF:male']},
 'ouch , stay safe ! hows the traffic there ?': {'label_about': ['ABOUT:male'],
  'label

In [16]:
with open('convai_all_labels.json','w') as f:
    json.dump(join_data,f)

# Alternativa: parsing solo con Stanza

In [17]:
with open('convai_all_labels.json','r') as f:
    data = json.load(f)

In [18]:
create_dependency_tags(data,'relative','_')

Comenzando parsing de dependencias....

Parsing de dependencias terminado. Duración del proceso: 3.1374217947324117 minutos
Comenzando a generar el encoding para las dependencias


100%|██████████| 73398/73398 [00:05<00:00, 12718.13it/s]

Proceso terminado con éxito


In [19]:
with open('convai_rel_complete.json','w') as f:
    json.dump(data,f)